In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251')

In [16]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
User-ID        1149780 non-null int64
ISBN           1149780 non-null object
Book-Rating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [3]:
books = pd.read_csv('data/BX-Books.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251')

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\Efim_Golovin\AppData\Local\Continuum\anaconda3\lib\site-packages\

In [17]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
ISBN                   271360 non-null object
Book-Title             271360 non-null object
Book-Author            271360 non-null object
Year-Of-Publication    271360 non-null object
Publisher              271358 non-null object
Image-URL-S            271360 non-null object
Image-URL-M            271360 non-null object
Image-URL-L            271357 non-null object
dtypes: object(8)
memory usage: 16.6+ MB


In [4]:
users = pd.read_csv('data/BX-Users.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251')

In [18]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
User-ID     278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [26]:
len(ratings[ratings['Book-Rating'] > 0]['User-ID'].values)

433671

In [27]:
len(ratings[ratings['Book-Rating'] > 0]['ISBN'].values)

433671

In [39]:
user_ids_dict = dict(enumerate(users['User-ID']))

In [40]:
user_ids_dict = dict(zip(user_ids_dict.values(), user_ids_dict.keys()))

In [41]:
user_ids_dict

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 11

In [42]:
book_isbns_dict = dict(enumerate(books['ISBN']))

In [43]:
book_isbns_dict = dict(zip(book_isbns_dict.values(), book_isbns_dict.keys()))

In [44]:
book_isbns_dict

{'0195153448': 0,
 '0002005018': 1,
 '0060973129': 2,
 '0374157065': 3,
 '0393045218': 4,
 '0399135782': 5,
 '0425176428': 6,
 '0671870432': 7,
 '0679425608': 8,
 '074322678X': 9,
 '0771074670': 10,
 '080652121X': 11,
 '0887841740': 12,
 '1552041778': 13,
 '1558746218': 14,
 '1567407781': 15,
 '1575663937': 16,
 '1881320189': 17,
 '0440234743': 18,
 '0452264464': 19,
 '0609804618': 20,
 '1841721522': 21,
 '1879384493': 22,
 '0061076031': 23,
 '0439095026': 24,
 '0689821166': 25,
 '0971880107': 26,
 '0345402871': 27,
 '0345417623': 28,
 '0684823802': 29,
 '0375759778': 30,
 '0425163091': 31,
 '3404921038': 32,
 '3442353866': 33,
 '3442410665': 34,
 '3442446937': 35,
 '0375406328': 36,
 '0446310786': 37,
 '0449005615': 38,
 '0060168013': 39,
 '038078243X': 40,
 '055321215X': 41,
 '067176537X': 42,
 '0061099686': 43,
 '0553582909': 44,
 '0671888587': 45,
 '0553582747': 46,
 '0425182908': 47,
 '042518630X': 48,
 '0440223571': 49,
 '0812523873': 50,
 '0842342702': 51,
 '0440225701': 52,
 '0